## 1. About the Dataset <a class="anchor" id="section1"></a>
- **Source**: The dataset used for experiments is Wiki-CS from the paper [*Wiki-CS: A Wikipedia-Based Benchmark for Graph Neural Networks*](https://arxiv.org/abs/2007.02901). 
- **Description**: It consists of nodes corresponding to Computer Science articles, with edges based on hyperlinks and 10 classes representing different branches of the field. 
- **Task**: Classify each node.

In [1]:
import pandas as pd, numpy as np

import time
import networkx as nx
import random
import igraph as ig

from sklearn.metrics import classification_report, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold

import igraph as ig
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
pio.templates.default = "plotly_white"



from sklearn.manifold import TSNE
from PIL import Image
import io
import os


import torch 
from torch_geometric.datasets import WikiCS
from torch_geometric.transforms import NormalizeFeatures, normalize_features
from torch_geometric.data import DataLoader
from torch.nn import Linear, ModuleList
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv

### 1.1. Global Statistics <a class="anchor" id="section11"></a>

In [2]:
# Load dataset 
# Original version had pre_transform=NormalizeFeatures(), but this led to worse performance for me (Jared)

dataset = WikiCS(root=os.path.join(os.getcwd(),'wikics'), is_undirected=False, force_reload=True)
data = dataset[0]

# Add the number of classes
data.num_classes = data.y.unique().shape[0]

Processing...
Done!


In [6]:
# Get some basic info about the dataset
# print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')
print(50*'=')  

# Gather some basic info about the graph
print(data)
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(50*'=')

# Gather some statistics about the graph
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')

Number of features: 300
Number of classes: 10
Data(x=[11701, 300], edge_index=[2, 297110], y=[11701], train_mask=[11701, 20], val_mask=[11701, 20], test_mask=[11701], stopping_mask=[11701, 20], num_classes=10)
Number of nodes: 11701
Number of edges: 297110
Average node degree: 25.39
Has isolated nodes: True
Has self-loops: True


### 1.2. Visualizing of the Graph <a class="anchor" id="section12"></a>

In [7]:
N_NODES = 500 # number of node to include

In [8]:
# Create a graph using igraph library
df_nodes = pd.DataFrame({
    'node_id': np.arange(data.num_nodes),
    'label' : data.y.cpu().detach().numpy()})

edges = [(data.edge_index[0,i].cpu().detach().numpy(), 
          data.edge_index[1,i].cpu().detach().numpy()) for i in range(data.num_edges)]
edges = [(int(e[0]), int(e[1])) for e in edges]

G = ig.Graph()
G.add_vertices(data.num_nodes)
G.add_edges(edges)

# Take a subset of the graph
G_subgraph = G.subgraph(list(G.vs)[:N_NODES]) if N_NODES > 0 else G

# Delete nodes that are not linked to any other node
nodes_edges = set([e[0] for e in G_subgraph.get_edgelist()]).union(set([e[1] for e in G_subgraph.get_edgelist()]))
G_subgraph.delete_vertices(list(set(G_subgraph.vs.indices) - nodes_edges))# https://igraph.org/python/tutorial/latest/tutorial.html#Layout%20algorithms

In [9]:
def plot_graph(graph, color_nodes, color_edges = 'grey', title = "", node_name="", layt_type = "fr"):
    
    """ Return a projection of the graph."""

    layt = G_subgraph.layout(layt_type) #kk #lgl #fr
    N = len(layt)

    width=800
    height=800

    Xn, Yn = {}, {}
    for label in df_nodes.label.unique():
        list_nodes = df_nodes[df_nodes.label==label].node_id.unique()
        Xn[label] = [layt[k][0] for k in range(N) if k in list_nodes]
        Yn[label] = [layt[k][1] for k in range(N) if k in list_nodes]

    Xe, Ye =[], []
    Xt, Yt =[], []

    for k, e in enumerate(G_subgraph.get_edgelist()):
        Xe+=[layt[e[0]][0],layt[e[1]][0], None]
        Ye+=[layt[e[0]][1],layt[e[1]][1], None]

    labels = df_nodes.label.unique()
    labels.sort()

    fig_data = []

    # edges
    fig_data += [
        go.Scatter(
            x=Xe, y=Ye, mode='lines',
            line= dict(color=color_edges, width=1), 
            name='link', showlegend=False, 
        )]

    # nodes
    fig_data += [
        go.Scatter(
            x=Xn[label], y=Yn[label], mode='markers', name='{} {}'.format(node_name, label),
            marker=dict(
                symbol='circle-dot', size=5, color=color_nodes[label],
                line=dict(color=color_nodes[label], width=0.5)),
            text="Label {}".format(label), hoverinfo='text') for label in labels]

    axis=dict(showline=False, # hide axis line, grid, ticklabels and  title
              zeroline=False,
              showgrid=False,
              showticklabels=False,
              title='')

    layout=go.Layout(title= title,
        font= dict(size=12),
        showlegend=True,
        autosize=False,
        width=width,
        height=height,
        xaxis=go.layout.XAxis(axis),
        yaxis=go.layout.YAxis(axis),
        margin=go.layout.Margin(l=40, r=40, b=85, t=100),
        hovermode='closest'
        )
    
    return go.Figure(data=fig_data, layout=layout)

In [10]:
# Pick a random color for every class
colors = px.colors.sequential.Plasma
random.shuffle(px.colors.sequential.Plasma)
color_nodes = {label : colors[i] for i, label in enumerate(df_nodes.label.unique())}

# Plot the figure
fig = plot_graph(data, color_nodes, color_edges = 'grey', 
                 title = "Network - Wiki CS", node_name = 'article', layt_type = "fr")

# Save the figure
#insights.save_plotly(id='graph_viz_wiki', figure = fig)

fig

## 2. Modeling  <a class="anchor" id="section2"></a>
The goal of this section is to compare two graph-based models (GCN and GAT) to the classical Random Forest classifier.

### 2.1. Building a First GCN Model  <a class="anchor" id="section21"></a>
> This section contains an implementation of a GCNs and an exploration of the results.

In [11]:
# Use GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
data = data.to(device)

In [12]:
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels, num_features, dropout_rate=0.5):
        super(GCN, self).__init__()
        torch.manual_seed(42)
        
        self.conv1 = GCNConv(num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, dataset.num_classes)
        
        self.p = dropout_rate

    def forward(self, x, edge_index):
        # First Message Passing Layer (Transformation)
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=self.p, training=self.training)

        # Second Message Passing Layer
        x = self.conv2(x, edge_index)

        # Output layer 
        return F.softmax(x, dim=1)

In [13]:
DROPOUT_RATE = 0.5
HIDDEN_CHANNELS = 8
learning_rate = 0.2
decay = 5e-4
N_EPOCHS = 500
E_PATIENCE = 100

In [14]:
# Initialize the model
model = GCN(hidden_channels=HIDDEN_CHANNELS, num_features = data.num_features).to(device)

# Initialize the optimizer
optimizer = torch.optim.Adam(model.parameters(), 
                             lr=learning_rate, 
                             weight_decay=decay)

# Define loss function
criterion = torch.nn.CrossEntropyLoss()

print(model)
print("Number of parameters: ", sum(p.numel() for p in model.parameters()))

GCN(
  (conv1): GCNConv(300, 8)
  (conv2): GCNConv(8, 10)
)
Number of parameters:  2498


In [15]:
def train(data, model, optimizer, criterion, i_split, train_mask):
    model.train()
    optimizer.zero_grad() 
    # Use all data as input, because all nodes have node features
    out = model(data.x, data.edge_index)  
    # Only use nodes with labels available for loss calculation
    loss = criterion(out[data.train_mask[:,i_split]], data.y[data.train_mask[:,i_split]])  
    loss.backward() 
    optimizer.step()
    
    return loss

def test(data, model, index, metric = 'accuracy'):
    index = index.cpu().detach().numpy()
    model.eval()
    
    out = model(data.x, data.edge_index)[index]
    
    if metric == 'auc':
        y_true = data.y[index]
        y_true = y_true.cpu().detach().numpy()
        y_true2 = np.zeros((y_true.shape[0], data.num_classes))
        
        for i in range(y_true.shape[0]):
            y_true2[i,y_true[i]] = 1
        score = roc_auc_score(y_true2, out.cpu().detach().numpy())
    else: 
        # Use the class with highest probability
        pred = out.argmax(dim=1)  
        correct = int((pred == data.y[index]).sum())
        score = correct / pred.shape[0]
    
    return score

def train_test(data, model, optimizer, criterion, metric = 'accuracy',
               i_split=0, train_mask = 'train_mask', test_mask = 'test_mask', val_mask='val_mask',
               n_epochs = 500, e_patience =100, min_acc= 10**(-5), verbose = True):
    t0 = time.time()
    losses = {}
    
    k = 0
    # Training loop
    for epoch in range(1, n_epochs):
        loss = train(data, model, optimizer, criterion, i_split=i_split, train_mask = train_mask)
        # Get the perf throughout the model training
        losses[epoch] = dict(
            loss = loss.cpu().detach().numpy(),
            train = test(data, model, data.train_mask[:,i_split], metric = metric),
            valid = test(data, model, data.val_mask[:,i_split], metric = metric),
            test = test(data, model, data.test_mask, metric = metric)
            
        )
        
        dela_t = time.time() - t0
        
        if verbose and (epoch % 10 == 0):
            print('Epoch: {}, Loss: {:.4f}, Train: {:.3f}, Valid: {:.3f}, Test: {:.3f} \
                  Time: {:.2f}s'.format(epoch, losses[epoch]['loss'], 
                                              losses[epoch]['train'], losses[epoch]['valid'], 
                                              losses[epoch]['test'], dela_t))
            
        # Enable early stopping
        if (epoch > 1) and abs(losses[epoch-1]['train']/losses[epoch-1]['train']-1) < min_acc :
            k += 1
        if k> e_patience:
            break
                  
    df_losses = pd.DataFrame.from_dict(
        losses, orient='index').reset_index().rename(columns={'index':'epoch'})
            
    return df_losses
                  
def visualize_loss(df_losses):
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_trace(go.Scatter(x=df_losses.epoch, y=df_losses.train, name = 'Train'),  secondary_y=True)
    fig.add_trace(go.Scatter(x=df_losses.epoch, y=df_losses.test, name = 'Test'), secondary_y=True)
    fig.add_trace(go.Scatter(x=df_losses.epoch, y=df_losses.valid, name = 'Valid'), secondary_y=True)
    fig.add_trace(go.Scatter(x=df_losses.epoch, y=df_losses.loss, name = 'Loss'))

    fig.update_yaxes(title_text="Loss", secondary_y=False)
    fig.update_yaxes(title_text="Accuracy", secondary_y=True)

    return fig

In [16]:
df_losses = train_test(data, model, optimizer, criterion, metric = 'auc',i_split=2,
                       n_epochs = N_EPOCHS, e_patience = E_PATIENCE)

Epoch: 10, Loss: 2.0687, Train: 0.647, Valid: 0.628, Test: 0.630                   Time: 1.22s
Epoch: 20, Loss: 2.0403, Train: 0.688, Valid: 0.665, Test: 0.664                   Time: 1.68s
Epoch: 30, Loss: 2.0088, Train: 0.710, Valid: 0.670, Test: 0.679                   Time: 2.13s
Epoch: 40, Loss: 2.0084, Train: 0.705, Valid: 0.665, Test: 0.673                   Time: 2.56s
Epoch: 50, Loss: 2.0101, Train: 0.702, Valid: 0.674, Test: 0.674                   Time: 2.98s
Epoch: 60, Loss: 1.9973, Train: 0.718, Valid: 0.678, Test: 0.685                   Time: 3.42s
Epoch: 70, Loss: 1.9588, Train: 0.733, Valid: 0.712, Test: 0.710                   Time: 3.85s
Epoch: 80, Loss: 1.8896, Train: 0.742, Valid: 0.707, Test: 0.711                   Time: 4.26s
Epoch: 90, Loss: 1.9277, Train: 0.738, Valid: 0.712, Test: 0.713                   Time: 4.68s
Epoch: 100, Loss: 1.8842, Train: 0.756, Valid: 0.721, Test: 0.721                   Time: 5.09s


In [17]:
fig = visualize_loss(df_losses)
fig

In [18]:
print('Training set score: {:.4f}'.format(test(data, model, data.train_mask[:,0], metric='auc')))
print('Valid set score: {:.4f}'.format(test(data, model, data.val_mask[:,0], metric='auc')))
print('Test set score: {:.4f}'.format(test(data, model, data.test_mask, metric='auc')))

out = model(data.x, data.edge_index)[data.test_mask]
pred = out.argmax(dim=1) 

print(classification_report(data.y[data.test_mask].detach().cpu().numpy(), pred.detach().cpu().numpy()))

Training set score: 0.7221
Valid set score: 0.7221
Test set score: 0.7170
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       147
           1       0.00      0.00      0.00       333
           2       0.55      0.83      0.66      1076
           3       0.84      0.74      0.79       966
           4       0.52      0.93      0.67      1339
           5       0.00      0.00      0.00       390
           6       0.00      0.00      0.00       206
           7       0.00      0.00      0.00       432
           8       0.00      0.00      0.00       246
           9       0.61      0.83      0.70       712

    accuracy                           0.59      5847
   macro avg       0.25      0.33      0.28      5847
weighted avg       0.43      0.59      0.49      5847



c:\Users\Jared\anaconda3\envs\pytorch\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

c:\Users\Jared\anaconda3\envs\pytorch\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

c:\Users\Jared\anaconda3\envs\pytorch\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



### 2.2. Comparing to a Traditional Classifier <a class="anchor" id="section22"></a>
> This section focuses on the comparison of the performance of the previous GCN with a Random Forest classifier.

#### ... cross-validation

In [19]:
def train_test_splits(GNN_model, model_params, learning_rate, decay, 
                     n_splits, n_epochs, e_patience, metric = 'accuracy', verbose=True):
    split_losses = {}

    for i_split in range(n_splits):
        print('Split : {}/{}'.format(i_split+1, n_splits))

        model = GNN_model(**model_params).to(device)

        optimizer = torch.optim.Adam(model.parameters(), 
                                     lr=learning_rate, 
                                     weight_decay=decay)

        criterion = torch.nn.CrossEntropyLoss()

        t0 = time.time()
        df_losses = train_test(data, model, optimizer, criterion, metric=metric, 
                               i_split = i_split,
                               n_epochs = n_epochs, e_patience =e_patience,
                               verbose = False)

        split_losses[i_split] = dict(
            train=df_losses['train'].iloc[-1], val=df_losses['valid'].iloc[-1],
            time = time.time() - t0
        )
        
        if verbose:
            print(split_losses[i_split])

    split_losses = pd.DataFrame.from_dict(split_losses, orient='index')
    split_losses = split_losses._append([split_losses.mean(axis=0), split_losses.std(axis=0)], ignore_index=True)
    split_losses.index = ['split_{}'.format(c+1) for c in range(n_splits)] + ['mean', 'std']
    
    return model, split_losses

In [20]:
model, split_losses = train_test_splits(
    GCN, model_params = {'hidden_channels': HIDDEN_CHANNELS, 'num_features' : data.num_features},
    learning_rate = learning_rate, decay = decay, metric ='auc', 
    n_splits = 2, n_epochs = N_EPOCHS, e_patience = E_PATIENCE
)

split_losses

Split : 1/2
{'train': 0.7613562170358444, 'val': 0.7451269521610857, 'time': 4.257864236831665}
Split : 2/2
{'train': 0.740676720700605, 'val': 0.7304908091081002, 'time': 5.045420169830322}


,train,val,time
split_1,0.761356,0.745127,4.257864
split_2,0.740677,0.730491,5.045420
mean,0.751016,0.737809,4.651642
std,0.014623,0.010349,0.556886


#### ... hyperparameter tunings

In [34]:
learning_rates = [0.1, 0.05, 0.01]
hidden_channels = [8, 16, 32]
dropout_rates = [0.25, 0.5]

## lr: 0.02, hc: 33, dp = 0.25

In [29]:
N_SPLITS = data.train_mask.shape[1]
N_SPLITS

20

In [30]:
def get_table(split_losses, params = {}):
    mean = split_losses.loc['mean']
    std = split_losses.loc['std']
    mean.index = [c+'_mean' for c in mean.index]
    std.index = [c+'_std' for c in std.index]
    
    table = pd.concat([mean, std], axis=0)
    
    if len(params)>0:
        for k,v in params.items():
            table[k] = v
    
    return table

In [35]:
from itertools import product

i = 0
for hc, lr, dp in product(hidden_channels, learning_rates, dropout_rates):
    
    print('Sc: {}/{}'.format(i+1, len(hidden_channels)*len(learning_rates)*len(dropout_rates)))
    print(f"Start training with hc: {hc} and lr: {lr}")
    
    params = {
        "GNN_model": GCN, 
        "model_params": {
            "hidden_channels": hc, 
            "num_features":data.num_features, 
            "dropout_rate": dp
        },
        "decay": decay, 
        "n_epochs": N_EPOCHS,
        "n_splits" : N_SPLITS, 
        "e_patience" : E_PATIENCE, 
        "metric":"auc",
        "learning_rate": lr
    }
    
    model_GCN, split_losses = train_test_splits(**params)
    
    table = get_table(split_losses, 
                      params = {"hc": hc, "lr": lr, "dp": dp, "params": str(params)})
    
    results = pd.concat([results, table], axis=1) if i>0 else table
    
    # updates best score and save best model
    if i  == 0:
        best_model_GCN = model_GCN
        best_score = split_losses.loc['mean']['val']
    else:
        if split_losses.loc['mean']['val'] > best_score:
            best_model_GCN = model_GCN
            best_score = split_losses.loc['mean']['val']
    i+=1
    
results_GCN = results.T['val_mean'].sort_values(ascending=False).reset_index(drop=True)

Sc: 1/18
Start training with hc: 8 and lr: 0.1
Split : 1/20
{'train': 0.7958141592259899, 'val': 0.771157514205831, 'time': 4.432183504104614}
Split : 2/20
{'train': 0.7705201875051513, 'val': 0.7604457079059957, 'time': 4.679542779922485}
Split : 3/20
{'train': 0.8104450308904548, 'val': 0.7723088801567675, 'time': 5.034269332885742}
Split : 4/20
{'train': 0.7415484676340343, 'val': 0.7406745299884341, 'time': 4.782484292984009}
Split : 5/20
{'train': 0.8187023178228356, 'val': 0.7789079523668925, 'time': 4.802876949310303}
Split : 6/20
{'train': 0.8001259410640387, 'val': 0.7754631292954606, 'time': 4.784903049468994}
Split : 7/20
{'train': 0.8211996097004522, 'val': 0.7879328597098636, 'time': 4.314106464385986}
Split : 8/20
{'train': 0.8002228313274629, 'val': 0.7775473663581873, 'time': 3.921318769454956}
Split : 9/20
{'train': 0.7573291636770018, 'val': 0.7315629292421623, 'time': 4.2030675411224365}
Split : 10/20
{'train': 0.7375219796288447, 'val': 0.7346488428359208, 'time': 4

In [36]:
results_GCN

0     0.759384
1     0.759194
2     0.756701
3     0.748538
4     0.734139
5     0.733882
6     0.729432
7     0.729349
8     0.726503
9      0.72543
10    0.723466
11    0.716302
12    0.714863
13     0.71327
14    0.708892
15    0.708242
16    0.706061
17     0.70565
Name: val_mean, dtype: object